In [1]:
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
import numpy as np
from keras.utils import np_utils

Using TensorFlow backend.


In [6]:
#Parsing a midi file
midi = converter.parse("./Downloads/Ml/Music Generation/midi_songs/EyesOnMePiano.mid")

In [7]:
midi

<music21.stream.Score 0x1669328a240>

In [29]:
#Printing the notes and chords
midi.show("text")

{0.0} <music21.stream.Part 0x166937fe0b8>
    {0.0} <music21.instrument.Piano 'Piano'>
    {0.0} <music21.tempo.MetronomeMark andantino Quarter=80.0>
    {0.0} <music21.key.Key of D major>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.note.Note F#>
    {0.5} <music21.note.Note A>
    {1.0} <music21.note.Note D>
    {3.75} <music21.note.Rest rest>
    {4.0} <music21.note.Note C>
    {4.5} <music21.note.Note A>
    {5.0} <music21.note.Note E>
    {7.75} <music21.note.Rest rest>
    {8.0} <music21.note.Note F#>
    {8.5} <music21.note.Note A>
    {9.0} <music21.note.Note D>
    {9.5} <music21.note.Note C#>
    {10.0} <music21.note.Note E>
    {10.5} <music21.note.Note D>
    {11.0} <music21.note.Note B>
    {11.5} <music21.note.Note C#>
    {12.0} <music21.note.Note A>
    {13.75} <music21.note.Rest rest>
    {14.0} <music21.chord.Chord C5 E5>
    {15.75} <music21.note.Rest rest>
    {16.0} <music21.note.Note F#>
    {16.5} <music21.chord.Chord E5 A5>
    {17.0} <music21.

In [9]:
# Flattening all the elements
elements_to_parse = midi.flat.notes
for e in elements_to_parse:
    print(e, e.offset)

<music21.note.Note A> 0.0
<music21.note.Note A> 0.0
<music21.note.Note A> 0.0
<music21.note.Note A> 0.25
<music21.note.Note G> 2/3
<music21.note.Note G> 2/3
<music21.note.Note F#> 1.0
<music21.note.Note F#> 1.25
<music21.note.Note D> 1.5
<music21.note.Note D> 1.5
<music21.note.Note C#> 1.75
<music21.note.Note C#> 1.75
<music21.note.Note A> 2.0
<music21.note.Note A> 2.25
<music21.note.Note G> 2.5
<music21.note.Note G> 2.5
<music21.note.Note F#> 2.75
<music21.note.Note F#> 2.75
<music21.note.Note D> 3.0
<music21.note.Note D> 3.25
<music21.note.Note A> 3.5
<music21.note.Note A> 3.5
<music21.note.Note D> 3.75
<music21.note.Note D> 3.75
<music21.note.Note A> 4.0
<music21.note.Note D> 4.0
<music21.note.Note B> 4.0
<music21.note.Note G> 4.0
<music21.note.Note A> 4.25
<music21.note.Note D> 4.25
<music21.note.Note B> 4.25
<music21.note.Note G> 4.25
<music21.note.Note E> 5.0
<music21.note.Note C#> 5.0
<music21.note.Note E> 5.25
<music21.note.Note C#> 5.25
<music21.note.Note F#> 17/3
<music21.not

In [33]:
try_notes = []

for ele in elements_to_parse:
    # If the element is a Note,  then store it's pitch
    if isinstance(ele, note.Note):
        try_notes.append(str(ele.pitch))
    
    # If the element is a Chord, split each note of chord and join them with +
    elif isinstance(ele, chord.Chord):
        try_notes.append("+".join(str(n) for n in ele.normalOrder))

In [34]:
isinstance(elements_to_parse[68], chord.Chord)

False

In [35]:
notes = []

for file in glob.glob("./Downloads/Ml/Music Generation/midi_songs/*.mid"):
    midi = converter.parse(file) # Convert file into stream.Score Object
    
    print("parsing %s"%file)
    
    elements_to_parse = midi.flat.notes
    
    
    for ele in elements_to_parse:
        # If the element is a Note,  then store it's pitch
        if isinstance(ele, note.Note):
            notes.append(str(ele.pitch))

        # If the element is a Chord, split each note of chord and join them with +
        elif isinstance(ele, chord.Chord):
            notes.append("+".join(str(n) for n in ele.normalOrder))

parsing ./Downloads/Ml/Music Generation/midi_songs\0fithos.mid
parsing ./Downloads/Ml/Music Generation/midi_songs\8.mid
parsing ./Downloads/Ml/Music Generation/midi_songs\ahead_on_our_way_piano.mid
parsing ./Downloads/Ml/Music Generation/midi_songs\AT.mid
parsing ./Downloads/Ml/Music Generation/midi_songs\balamb.mid
parsing ./Downloads/Ml/Music Generation/midi_songs\bcm.mid
parsing ./Downloads/Ml/Music Generation/midi_songs\BlueStone_LastDungeon.mid
parsing ./Downloads/Ml/Music Generation/midi_songs\braska.mid
parsing ./Downloads/Ml/Music Generation/midi_songs\caitsith.mid
parsing ./Downloads/Ml/Music Generation/midi_songs\Cids.mid
parsing ./Downloads/Ml/Music Generation/midi_songs\cosmo.mid
parsing ./Downloads/Ml/Music Generation/midi_songs\costadsol.mid
parsing ./Downloads/Ml/Music Generation/midi_songs\dayafter.mid
parsing ./Downloads/Ml/Music Generation/midi_songs\decisive.mid
parsing ./Downloads/Ml/Music Generation/midi_songs\dontbeafraid.mid
parsing ./Downloads/Ml/Music Generatio

In [15]:
len(notes)

60498

In [16]:
with open("notes", 'wb') as filepath:
    pickle.dump(notes, filepath)

with open("notes", 'rb') as f:
    notes= pickle.load(f)

n_vocab = len(set(notes))

print("Total notes- ", len(notes))
print("Unique notes- ",  n_vocab)

Total notes-  60498
Unique notes-  359


In [36]:
# Hoe many elements LSTM input should consider
sequence_length = 100 #assuming to be 100

# All unique classes
pitchnames = sorted(set(notes))

# Mapping between ele to int value
ele_to_int = dict( (ele, num) for num, ele in enumerate(pitchnames) )

network_input = []
network_output = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i : i+sequence_length] # contains 100 values
    seq_out = notes[i + sequence_length] #after iteration containg next 100 values including seq_out and removing 0 element of seq_in
    
    network_input.append([ele_to_int[ch] for ch in seq_in])
    network_output.append(ele_to_int[seq_out])

In [19]:
# No. of examples
n_patterns = len(network_input)
print(n_patterns)

# Desired shape for LSTM
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
print(network_input.shape)

60398
(60398, 100, 1)


In [20]:
normalised_network_input = network_input/float(n_vocab)

# Network output are the classes, encode into one hot vector
network_output = np_utils.to_categorical(network_output)

print(normalised_network_input.shape)
print(network_output.shape)

(60398, 100, 1)
(60398, 359)


In [21]:
from keras.models import Sequential, load_model
from keras.layers import *

In [22]:
model = Sequential()
model.add( LSTM(units=512,
               input_shape = (normalised_network_input.shape[1], normalised_network_input.shape[2]),
               return_sequences = True) )
model.add( Dropout(0.3) )
model.add( LSTM(512, return_sequences=True) )
model.add( Dropout(0.3) )
model.add( LSTM(512) )
model.add( Dense(256) )
model.add( Dropout(0.3) )
model.add( Dense(n_vocab, activation="softmax") )

In [24]:
model.compile(loss="categorical_crossentropy", optimizer="adam")

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)              

In [26]:
#Already trained on google colab but other computation was done on local machine
model = load_model("./Downloads/Ml/Music Generation/new_weights.hdf5")

## Predictions

In [27]:
sequence_length = 100
network_input = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i : i+sequence_length] # contains 100 values
    network_input.append([ele_to_int[ch] for ch in seq_in])

In [28]:
# Any random start index
start = np.random.randint(len(network_input) - 1)

# Mapping int_to_ele
int_to_ele = dict((num, ele) for num, ele in enumerate(pitchnames))

# Initial pattern 
pattern = network_input[start]
prediction_output = []

# generate 200 elements
for note_index in range(200):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1)) # convert into numpy desired shape 
    prediction_input = prediction_input/float(n_vocab) # normalise
    
    prediction =  model.predict(prediction_input, verbose=0)
    
    idx = np.argmax(prediction)
    result = int_to_ele[idx]
    prediction_output.append(result) 
    
    # Remove the first value, and append the recent value.. 
    # This way input is moving forward step-by-step with time..
    pattern.append(idx)
    pattern = pattern[1:]

print(prediction_output)

['D5', 'C5', 'B-4', '10+2+5', 'A5', 'G5', 'F5', '2+5+9', 'E5', 'D5', 'C5', '0+4+7', 'D5', 'E5', '2+5+9', 'D5', '2+5+9', 'E5', 'F5', 'G5', '0+4+7', 'G5', 'A5', '5+9+0', 'G5', 'F5', 'E5', '0+4+7', 'E5', 'D5', '2+5+9', 'E5', 'F5', 'B-4', '10+2+5', 'B-4', 'A4', '11+2+5', 'A5', 'G5', 'F5', '10+2+5', 'E5', 'C5', 'D5', '2+5+9', 'E5', 'F5', 'G5', '0+4+7', 'G5', 'A5', '5+9+0', 'G5', 'F5', 'E5', '0+4+7', 'D5', 'C5', 'B-4', '10+2+5', 'A5', 'G5', 'F5', '2+5+9', 'E5', 'D5', 'C5', '0+4+7', 'D5', 'E5', '2+5+9', 'D5', '2+5+9', 'E5', 'F5', 'G5', '0+4+7', 'G5', 'A5', '5+9+0', 'G5', 'F5', 'E5', '0+4+7', 'E5', 'D5', '2+5+9', 'E5', 'F5', 'B-4', '10+2+5', 'B-4', 'A4', '11+2+5', 'A5', 'G5', 'F5', '10+2+5', 'E5', 'C5', 'D5', '2+5+9', 'E5', 'F5', 'G5', '0+4+7', 'G5', 'A5', '5+9+0', 'G5', 'F5', 'E5', '0+4+7', 'D5', 'C5', 'B-4', '10+2+5', 'A5', 'G5', 'F5', '2+5+9', 'E5', 'D5', 'C5', '0+4+7', 'D5', 'E5', '2+5+9', 'D5', '2+5+9', 'E5', 'F5', 'G5', '0+4+7', 'G5', 'A5', '5+9+0', 'G5', 'F5', 'E5', '0+4+7', 'E5', 'D5',